In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, LinearConstraint, Bounds
from scipy.integrate import solve_ivp

# Calculate "Adiabats" in the Fo-SiO$_2$ system

This notebook derives and calculates self-gravitating isentropic profiles for a 1-D solid-state upwelling problem with no phase separation.

##  Model Derivation

Conservation of Mass, Composition, and Entropy along with lithostatic pressure in  a 1-D steady state upwelling column for $N$ phases all traveling at the same upwelling velocity $W$ can be written

$$
    \frac{d\ }{dz} \rho_i \phi_i W = \Gamma_i 
$$

$$
    \frac{d\ }{dz} \rho_i \phi_i W c_i^k = \Gamma_i^k 
$$

$$
    \frac{d\ }{dz} \bar{\rho} W = 0
$$

$$
   \frac{d\ }{dz} \sum_i \rho_i \phi_i s_i W  = 0
$$

$$
    \frac{d P}{dz} = -\bar{\rho} g
$$

Where $\rho_i$ is the intrinsic density of phase $i$ ($i = 0,\ldots,N-1$), $\phi_i$ is the volume fraction and .  $c_i^k$ is the mass fraction of endmember $k$ in phase $i$ and $\Gamma_i^k(T,P,\Phi,C)$ is the net mass-transfer rate of component $k$ in phase $i$ over all reactions.  

Because 

$$
\sum_k c_i^k = 1
$$ 

it follows that 

$$
\Gamma_i(T,P,\Phi,C) = \sum_k \Gamma_i^k(T,P,\Phi,C)
$$

is the net mass transfer of phase $i$ over all reactions. 

Moreover by construction and definition

$$
    \sum_i \Gamma_i = 0\quad \mathrm{and} \quad \bar{\rho} = \sum_i \rho_i\phi_i
$$

is the mean density of the $N$ phase assemblage.  Finally $s_i$ is the specific entropy of each phase and the fourth equation is simply a statment of total entropy conservation.

In total, given that $\rho_i$, $\Gamma_i^k$, $s_i$ are thermodynamic functions of $(T,P,C)$  given by the ``ThermoCodegen`` reaction objects, the actual unknowns in this system are $\phi_i$, $W$, $c_i^k$, $T$,  and $P$

## A useful change of variables

If we define the mass fraction of phase $i$ as

$$
    F_i = \frac{\rho_i\phi_i}{\bar{\rho}}
$$ 

we can remove the dependence on $W$ and rewrite the system of equations in terms of the variables $F_i$, $c_i^k$, $s_i$, and $P$.  

Integrating the third equation and applying the boundary condition at $z=0$ gives

$$
    \bar{\rho}W = \bar{\rho}_0 W_0
$$

where the RHS is the mean density and upwelling velocity evaluated at $z=0$.  With these definitions it follows that

$$
    \rho_i\phi_i W = \bar{\rho}_0W_0 F_i
$$ 

and we can rewrite the system of equations in terms of $\mathbf{F}$ as 

$$
    \frac{d\ }{dz} F_i = \frac{1}{\bar{\rho}_0W_0}\Gamma_i
$$

$$
    \frac{d\ }{dz} F_i c_i^k = \frac{1}{\bar{\rho}_0W_0}\Gamma_i^k 
$$

$$
   \frac{d\ }{dz} \sum_i F_i s_i  = 0
$$

$$
    \frac{d P}{dz} = -\left(\sum_i \frac{F_i}{\rho_i}\right)^{-1} g
$$


Where it is relatively straightforward to show that an alternative definition of the mean density is

$$
    \bar{\rho} = \left(\sum_i \frac{F_i}{\rho_i}\right)^{-1}
$$
    

## Temperature Equations

As written,  Temperature is an implicit function of the entropy conservation equation which could be integrated directly to show

$$
    \sum_i F_i s_i = S_0
$$ 

such that total entropy is conserved.  Written in this form, the system becomes a DAE (Differential Algebraic Equation) which can be solved using more sophisticated solvers,  however, using standard relationships for derivatives of entropy and applying the product rule to both the compositional equations and the entropy equation, we can rewrite the full system as a pure ODE of the form

$$
    \frac{d\mathbf{u}}{dt} = \mathbf{f}(t,\mathbf{u})
$$

where 
$$
    \mathbf{u} = \begin{bmatrix} \mathbf{F} \\ \mathbf{C} \\ T \\ P \\\end{bmatrix}
$$

Given the Thermodynamic relationships

$$
    \frac{\partial s_i}{\partial T} = \frac{c_{pi}}{T}, \quad \frac{\partial s_i}{\partial P} = -\frac{\alpha_i}{\rho_i}
$$ 

and using the product rule, the system of equations in temperature form becomes


$$
    \frac{d\ }{dz} F_i = \frac{1}{\bar{\rho}_0W_0}\Gamma_i
$$

$$
    \frac{d\ }{dz} c_i^k = \frac{1}{\bar{\rho}_0W_0(F_i + \epsilon)}\left[\Gamma_i^k - c_i^k\Gamma_i\right] 
$$

$$
   \frac{d T }{dz}  =  \frac{T}{\bar{c_p}}\left[ -\bar{\alpha}g + \frac{1}{\bar{\rho}_0W_0} \sum_i \left(-s_i\Gamma_i + \sum_k \frac{\partial s_i}{\partial c_i^k}\left(\Gamma_i^k - c_i^k\Gamma_i\right)\right)\right]
$$

$$
    \frac{d P}{dz} = -\left(\sum_i \frac{F_i}{\rho_i}\right)^{-1} g
$$
    
    
where 
$$
    \bar{c_p} = \sum_i F_ic_{pi},\quad \bar{\alpha} = \frac{\sum_i (F_i/\rho_i)\alpha_i}{\sum_i F_i/\rho_i}
$$

are respectively, the mean heat capacity and mean thermal expansivity.  $\epsilon$ is a small regularization parameter to handle the singularity in compositions that arise when a phase arises or disappears (i.e. $F_i=0$)

## Scaling

For numerical reasons, it is useful to non-dimensionalize the equations. A useful scaling for  a 1-D steady state upwelling column is

$$
\begin{matrix}
     T = T_0T' & P = P_0 P' & z = \frac{P_0}{\bar{\rho}_0 g}z' & \Gamma = r_0\sigma_0\Gamma'  \\
\end{matrix}
$$

where $T_0, P_0$ is the pressure at the base of the column, $\bar{\rho}_0$ is the mean density at $T_0$, $P_0$ thus

$$
    h = \frac{P_0}{\bar{\rho}_0 g}
$$

is an *estimate* of the depth of the melting column (and would be the depth if densities were constant).  In general the actual depth needs to be calculated from the integration of density.  $r_0$ is the reaction rate in mass per unit area per time and  $\sigma_0$ is the  specific surface area available for reaction.  The product $r_0\sigma_0$ has units of density per time.  

Substituting into the dimensional equations and dropping primes yields the final system of ODE's we will solve

$$
    \frac{d F_i }{dz}  = Da\Gamma_i
$$

$$
    \frac{d c_i^k}{dz}  = \frac{Da}{(F_i + \epsilon)}\left[\Gamma_i^k - c_i^k\Gamma_i\right] 
$$

$$
   \frac{d T }{dz}  =  \frac{T}{\bar{c_p}}\left[ -\frac{P_0}{\bar{\rho}_0}\bar{\alpha} + Da\sum_i \left(-s_i\Gamma_i + \sum_k \frac{\partial s_i}{\partial c_i^k}\left(\Gamma_i^k - c_i^k\Gamma_i\right)\right)\right]
$$

$$
    \frac{d P}{dz} = -\frac{\bar{\rho}}{\bar{\rho}_0}
$$

where 
$$
    Da = \frac{P_0r_0\sigma_0}{\bar{\rho}_0^2g W_0}
$$ 

is the Dahmköler number which is a measure of the degree of disequilibrium during transport. $Da=0$ is complete disequilibrium and $Da\rightarrow\infty$ implies equilibrium.


## 1-D isentropes in the simplified mafic Fo-SiO2 system

Here we consider a reduced version in the mafic end-members of the Fo-SiO2 system that is appropriate to upper mantle conditions.  

To view the full system we first import the `ThermoCodegen` reaction object (which needs to be in the python path) appropriate for this system


In [ ]:
import py_fo_sio2_poly_linear_rxns as pfs
rxn = pfs.fo_sio2_poly_linear_rxns()
rxn.report()

Which currently includes 4 phases and 3 reactions although for problems with no initial SiO2 phase we expect we can ignore the Silica phase.

For convenience we will identify some indices for phases and components

In [ ]:
iLq = 0
iOl = 1
iOpx = 2
iSi = 3
kSi = 0
kFo = 1

### Some utility functions for converting between compositions and Temperature units

In [ ]:
# load liquid composition vector given just x_si204
set_xliq = lambda xq : np.array([xq,1.-xq])

# convert mole fractions between moles of Si2O4 and SiO2
si2_to_si = lambda xq : 2.*xq/(1.+xq)
si_to_si2 = lambda xq : xq/(2.-xq)

# convert temperatures
to_Kelvin = lambda T : T + 273.15
to_Celsius = lambda T : T - 273.15

### Create right hand side function of ODEs

In [ ]:
def f(t,u,rxn,scale,epsilon=1.e-3,verbose=False):
    '''
    return rhs of the dimensionless 1-D isentropic upwelling equations
    
    Parameters
    ----------
    
    t: float
        time
    u: array
        solution array [ F, c_(Lq)^{Si2O4}, T, P ]
    rxn:  ThermoCodegen Reaction object
    scale: dict
        dictionary containing scales for T,P,rho
    epsilon: float
        regularization parameter for composition equation
        
    
    '''
    # Extract variables
    N = 4
    F = u[:N]
    csi = u[N]
    T = u[N+1]
    P = u[N+2]
    
    # scale Temperature and pressure
    
    Ts = scale['T']*T
    Ps = scale['P']*P
    
    # set the melt composition
    C[iLq] = np.array([csi,1.-csi])
    
    # calculate thermodynamic properties from the rxn object
    gamma_i = np.array(rxn.Gamma_i(Ts,Ps,C,F))
    gamma_ik_lq = np.array(rxn.Gamma_ik(Ts,Ps,C,F)[iLq])
    if verbose:
        print(gamma_i)
        print(gamma_ik_lq)
    
    # liquid properties
    dCik_lq = gamma_ik_lq - np.array(C[iLq])*gamma_i[iLq]
    dsdc_lq = np.array(rxn.ds_dC(Ts,Ps,C)[iLq])
    if verbose:
        print(dCik_lq)
        print(dsdc_lq)
    
    # phase properties
    rho = np.array(rxn.rho(Ts, Ps, C))
    alpha = np.array(rxn.alpha(Ts,Ps,C))
    cp = np.array(rxn.Cp(Ts, Ps , C))
    s = np.array(rxn.s(Ts, Ps, C))
    v = F/rho
    
    # mean properties
    rho_bar = 1./sum(v)
    alpha_bar = v.dot(alpha)*rho_bar
    cp_bar = F.dot(cp)
    s_bar = F.dot(s)
    
    A = scale['P']/scale['rho']
    
    dFdz = gamma_i
    dcsidz = (gamma_ik_lq[kSi] - csi*gamma_i[iLq])/(F[iLq] + epsilon)
    dTdz = T/cp_bar * ( -A*alpha_bar  - s.dot(gamma_i) + dsdc_lq.dot(dCik_lq) )
    dPdz = -rho_bar/scale['rho']
    
    if verbose:
        print(T/cp_bar, A*alpha_bar, s.dot(gamma_i), dsdc_lq.dot(dCik_lq))
    du = np.empty(u.shape)
    du[:N] = dFdz
    du[N:] = np.array([dcsidz, dTdz, dPdz])
    return du
    
    

### Set Initial conditions

Temperature and pressure

In [ ]:
# initial temperature and pressure in K and bars
T0 = to_Kelvin(1680.)
P0 = 11000.

Approximate Melt composition

We will start by assuming the initial liquid is pure liquid Opx which in liquid endmember space is 1/4 mols of Qz4 and 1/2 mol of Fo or `n[iLq] = [ 0.25, 0.5 ]` or `X[iLq] = [1/3, 2/3]`.  This will be useful for calculating initial mass fractions given densities, and then we will refine our initial disequilibrium melt concentration below

In [ ]:
# initialize Mol fraction matrix 
X0 = rxn.zero_C()
for i in range(1,len(X0)):
    X0[i] = [1.]
X0[iLq] = [1/3.,2/3.]
print('X = ',X0)

C0 = rxn.X_to_C(X0)
print('C = ',C0)

Lq = pfs.Liquid()
print('Liquid Formula = {} (4 oxygen basis)'.format(Lq.formula(T0,P0,X0[iLq])))


Initial Mass fractions

We start with initial volume fractions, then convert them to mass fractions given the densities of the phases

In [ ]:
Phi0  = np.array([0., .6, .4, 0.])
rho = np.array(rxn.rho(T0,P0,C0))
F0 = rho*Phi0/rho.dot(Phi0)

### Set the Da number

Here we will actually build the Dahmkohler number into the reaction object by setting internal parameters.  For every reaction object there are sets of parameters that can be set internally at run time.  To view them use

In [ ]:
rxn.list_parameters()

where `Stot` is the available surface area and `r0_melt` and `r0_xtal` are the forward and backwards reaction rates.  Here they are both set to 1 so the reaction rates $\Gamma$ are already dimensionless.  To scale all the reactions by the Da number we can simply set `Stot=Da`

In [ ]:
Da = 100.
rxn.set_parameter('Stot',Da)
rxn.list_parameters()

### Solve for initial melt concentration

The first incipient disequilbrium liquid at any P,T should satisfy

$$
    \Gamma_{Lq}^k(T,P,C,\Phi) - c_{Lq}^k\Gamma_{Lq}(T,P,C,\Phi) = 0
$$ 

which is a non-linear problem in $C$ (assuming $F_{Lq}=0$). However,  it should be cast as a non-linear constrained optimization problem with the additional constraints that $\mathbf{1}^T\mathbf{c}_{Lq}=1.$ and all concentrations are bounded in $[0,1]$

#### Calculate the initial  liquid composition


In [ ]:
def func(cLq,T,P,C,F):
    """ function to return || MdC/dT|| """
    C[iLq] = cLq
    res = np.array(rxn.Gamma_ik(T,P,C,F)[iLq]) - np.array(C[iLq])*rxn.Gamma_i(T,P,C,F)[iLq]
    return np.linalg.norm(res)

#constraints and bounds
lc = LinearConstraint(np.ones(2),1.,1.)
bnds = Bounds(np.zeros(2),np.ones(2))

Run the optimizer

In [ ]:
c0 = C0[iLq].copy()
sol = minimize(func, c0, args=(T0, P0, C0, F0),bounds=bnds,constraints=lc)
print(sol.message)
print('residual = {}, sum(cf)= {}'.format(sol.fun,np.sum(sol.x)))
print('Liquid Composition = {}'.format(sol.x))
C0[iLq] = sol.x
C = C0.copy()

### Initialize Density and scaling

In [ ]:
rho = np.array(rxn.rho(T0, P0, C0))
v = F0/rho
rho_bar = 1/sum(v)

# depth/distance scale in km
g = 9.81 # m/s^2
h = P0*1e5/(rho_bar*100.*g)/1000

scale= {'T':T0, 'P':P0, 'rho':rho_bar, 'h':h}
print(scale)



#### Set epsilon and initial condition u0

In [ ]:
epsilon = 1.e-3
N = len(F0)
u0=np.empty(N+3)
u0[:N] = F0
u0[N:] = np.array([ C0[iLq][kSi], 1., 1.])

### Solve the system of ODE's

Here we will solve the system of ODE's from P0 to the surface, by setting an event which terminates the solution when the pressure reaches Pmin = 1 bar

In [ ]:
Pmin = 1
event = lambda t, y, rxn, scale: P0*y[-1]-Pmin
event.terminal = True

sol = solve_ivp(f,[0,1.1],u0,args=(rxn,scale),dense_output=True,method='Radau',rtol=1.e-6,atol=1.e-10, events=event)
print('{} P_end = {} bar.  Used {} steps: '.format(sol.message, sol.y[-1][-1]*scale['P'], len(sol.t)))


#### and plot

In [ ]:
z = np.linspace(0,1,100)
depth = -scale['h']*(1 - z)
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,3,1)
ax.plot(sol.sol(z)[:N].T,depth)
ax.set_xlabel('F')
ax.set_ylabel('depth (km)')
ax.legend(['Lq', 'Ol', 'Opx', 'Qz'],loc='best')
ax.grid()

ax = fig.add_subplot(1,3,2)
ax.plot(sol.y[N].T,-scale['h']*(1 - sol.t),'o-')
ax.set_xlabel('cSi_lq')
ax.set_title('Da = {}'.format(Da))
ax.grid()

ax = fig.add_subplot(1,3,3)
ax.plot(to_Celsius(sol.sol(z)[-2].T*scale['T']),depth)
ax.set_xlabel('T (C)')
ax.grid()
plt.show()


#### Plot thermodynamic properties

In [ ]:
F = sol.sol(z)[:N].T
cfsi = sol.sol(z)[N]
T = scale['T']*sol.sol(z)[-2]
P = scale['P']*sol.sol(z)[-1]

gamma_i = np.zeros(F.shape)
s = gamma_i.copy()
rho = gamma_i.copy()
stot = np.zeros(T.shape)

for i,t in enumerate(T):
    C[iLq]  = [ cfsi[i], 1. - cfsi[i]]
    gamma_i[i] = rxn.Gamma_i(T[i],P[i],C,F[i])
    s[i]       = rxn.s(T[i],P[i],C)
    rho[i]     = rxn.rho(T[i], P[i], C)
    stot[i] = F[i].dot(s[i])

s_err = np.linalg.norm((stot - stot[0]))/stot[0]

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,3,1)
ax.plot(gamma_i,depth)
ax.set_xlabel('$\Gamma_i$')
ax.set_ylabel('depth (km)')
ax.legend(['Lq', 'Ol', 'Opx', 'Qz'],loc='best')
ax.grid()

ax = fig.add_subplot(1,3,2)
ax.plot(s,depth)
ax.plot(stot,depth,'k--')
ax.set_xlabel('$s$')
ax.set_ylabel('depth (km)')
ax.legend(['Lq', 'Ol', 'Opx', 'Qz', '$S_{tot}$'], loc='best')
ax.set_title('$\Delta S/S_0$= {}'.format(s_err))
ax.grid()

ax = fig.add_subplot(1,3,3)
ax.plot(rho[:,:3]*100,depth)
ax.set_xlabel('$\\rho$ kg/m$^3$')
ax.legend(['Lq', 'Ol', 'Opx', 'Qz'],loc='best')
ax.grid()
plt.show()
